<a href="https://colab.research.google.com/github/StuteePatil/Masters-Dissertation/blob/master/ML_for_Codon_Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Uploading the GZ file downloaded from Saccharomyces Genome Database (SGD)
from google.colab import files
file = files.upload()

Saving orf_coding_all.fasta.gz to orf_coding_all.fasta.gz


In [2]:
# unzipping the GZ file to obtain FASTA file that contains sequence data for Yeast protein coding sequences
import gzip
import shutil

with gzip.open('orf_coding_all.fasta.gz', 'r') as f:
    file_content = f.read()

with gzip.open('orf_coding_all.fasta.gz', 'rb') as f_in:        # opens the GZ file
    with open('orf_coding_all.fasta', 'wb') as f_out:           # unzips the GZ file to retrieve FASTA file
        shutil.copyfileobj(f_in, f_out)

# convert the file to a utf-8 format for readability
f = file_content.decode("utf-8", "strict")  

In [3]:
# installing biopython package used for reading biological sequence data
!pip install biopython

from Bio import SeqIO
import math
import csv

# parsing FASTA file using a Biopython API to retrieve sequence data
dna_sequences = SeqIO.parse("orf_coding_all.fasta", "fasta")

# creating a CSV file for storing the sequence data read from FASTA file
f = open('yeast_cds.csv','w', newline='')
writer = csv.writer(f)

header = ['ID', 'Description', 'Coding Sequence']       
writer.writerow(header)                     # creates headers for CSV file

# iteratively read individual sequence data and store in Excel sheet
for cur_record in dna_sequences:
    data = []
    
    data.append(cur_record.name)            # retrieves sequence name, denoted by a unique ID
    data.append(cur_record.description)     # retrieves description of the sequence that follows sequence ID
    data.append(cur_record.seq)             # retrieves nucleotide sequence

    writer.writerow(data)                   # appends the data to CSV file
f.close()

In [4]:
# function to convert a nucleotide sequence into a sequence of codons 
# by grouping three consecutive nucleotides to form a codon

clen = 3

def codon(seq):
    codon_lst = []
    for i in range(len(seq)//3):
        cd = seq[i*clen : (i+1)*clen]
        codon_lst.append(cd)
    return codon_lst

In [5]:
# function to translate a sequence of codons into the corresponding sequence of amino acids

def translate(seq): 
    # dictionary that maps each codon to corresponding amino acid it codes for
    table = { 
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M', 
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T', 
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K', 
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L', 
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P', 
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q', 
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R', 
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V', 
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A', 
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E', 
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G', 
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S', 
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L', 
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_', 
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W', 
    } 
    
    protein = [] 
    for i in seq:
        protein.append(table[i])  
    return protein 

In [6]:
import pandas as pd
# load Yeast data and filter to get only coding sequences
data = pd.read_csv('yeast_cds.csv')

# extracting only sequence data
df = data['Coding Sequence']
print(df.head(10))

0    ATGGTACTGACGATTTATCCTGACGAACTCGTACAAATAGTGTCTG...
1    ATGGAGCAAAATGGCCTTGACCACGACAGCAGATCTAGCATCGATA...
2    ATGGCATCCACCGATTTCTCCAAGATTGAAACTTTGAAACAATTAA...
3    ATGGGTGTCACCAGCGGTGGCCTTAACTTCAAAGATACCGTCTTCA...
4    ATGTCAAAAGCTGTCGGTATTGATTTAGGTACAACATACTCGTGTG...
5    ATGATCAAATCTACAATTGCTCTACCCTCTTTCTTCATTGTTTTAA...
6    ATGACTTTGGCTTTTAATATGCAACGGTTGGTGTTTCGTAATTTGA...
7    ATGGAGCCAGAGAGCATAGGCGATGTGGGGAACCATGCCCAGGATG...
8    ATGCTACCCTATATGGACCAAGTACTAAGGGCATTTTATCAGAGCA...
9    ATGCCTGCTGTCTTGAGAACCAGGTCCAAAGAATCCTCTATAGAGC...
Name: Coding Sequence, dtype: object


In [7]:
# converting a nucleotide sequence to a sequence of codons and amino acids
aa_seq = []
cd_seq = []

for seq in df:
    cds = codon(seq)                # converts 'seq' to sequence of codons
    cd_seq.append(cds)
    aa_seq.append(translate(cds))   # resulting sequence of codons, 'cds', is translated to a sequence of amino acids

print(len(aa_seq))
print(len(cd_seq))

6713
6713


In [8]:
# splitting Yeast data for training and validation
train_size = int(len(aa_seq) * 0.8)     # 80% of the Yeast sequences are used for training, and the rest 20% are used for validation

train_aas = aa_seq[:train_size]         # amino acid sequences used as input training data
train_cds = cd_seq[:train_size]         # codon sequences used as output training data

val_aas = aa_seq[train_size:]           # amino acid sequences used as input validation data
val_cds = cd_seq[train_size:]           # codon sequences used as output validation data

print('Train:', len(train_aas), len(train_cds))
print('Test:', len(val_aas), len(val_cds))

Train: 5370 5370
Test: 1343 1343


In [9]:
import itertools

# identifies all unique amino acids in the sequence data
vocab = set(itertools.chain(*[[w for w in s] for s in train_aas])) 

# identifies all unique codons in the sequence data
tags = set(itertools.chain(*[[w for w in s] for s in train_cds]))

sentenecs_lens = map(len, train_aas)

# prints number of unique amino acids and codons respectively
print('Number of amino acids: ', len(vocab))        # includes 20 amino acids + stop codons = 21
print('Number of codons: ', len(tags))              # 64 possible nucleotide triplets

Number of amino acids:  21
Number of codons:  64


In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros

MAX_LEN = 2000              # standard value of sequence length considered for model development

# Tokenize amino acid sequences (input sequences)
VOCAB_SIZE = len(vocab)     # gives the number of unique amino acids, which is 21

words_tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters=[])           # define a tokeniser for pre-processing amino acid sequences
words_tokenizer.fit_on_texts(map(lambda s: ' '.join(s), train_aas))
word_index = words_tokenizer.word_index                                 # integer encoding of each type of amino acid to a unique integer value
word_index['__PADDING__'] = 0                                           # 0 value is assigned to dummy used for padding sequences
index_word = {i:w for w, i in word_index.items()}                       # reverse mapping of integers to amino acids
print('Unique tokens:', (word_index))

train_sequences = words_tokenizer.texts_to_sequences(map(lambda s: ' '.join(s), train_aas))     # tokenising input training data
val_sequences = words_tokenizer.texts_to_sequences(map(lambda s: ' '.join(s), val_aas))         # tokenising input validation data

train_sequences_padded = pad_sequences(train_sequences, maxlen=MAX_LEN)     # padding input training data to obatin sequences of length MAX_LEN = 2000
val_sequences_padded = pad_sequences(val_sequences, maxlen=MAX_LEN)         # padding input validation data to obatin sequences of length MAX_LEN = 2000

print(train_sequences_padded.shape, val_sequences_padded.shape)

Unique tokens: {'l': 1, 's': 2, 'k': 3, 'i': 4, 'e': 5, 'n': 6, 't': 7, 'd': 8, 'v': 9, 'a': 10, 'g': 11, 'f': 12, 'r': 13, 'p': 14, 'q': 15, 'y': 16, 'h': 17, 'm': 18, 'c': 19, 'w': 20, '_': 21, '__PADDING__': 0}
(5370, 2000) (1343, 2000)


In [11]:
# Tokenize codon sequences (output sequences)
import numpy as np

TAG_SIZE = len(tags)        # gives the number of unique codons, which is 64

tags_tokenizer = Tokenizer(num_words= TAG_SIZE, filters='', lower=False)    # define a tokeniser for pre-processing codon sequences
tags_tokenizer.fit_on_texts(map(lambda s: ' '.join(s), train_cds))
tag_index = tags_tokenizer.word_index                                       # integer encoding of each type of codon to a unique integer value
tag_index['__PADDING__'] = 0                                                # 0 value is assigned to dummy used for padding sequences
index_tag = {i:w for w, i in tag_index.items()}                             # reverse mapping of integers to codons

index_tag_wo_padding = dict(index_tag)
index_tag_wo_padding[tag_index['__PADDING__']] = '0'                        # reverse mapping of integer encoding for padding dummy to '0'
print('Unique tags:', (tag_index))

train_tags = tags_tokenizer.texts_to_sequences(map(lambda s: ' '.join(s), train_cds))   # tokenising output training data
val_tags = tags_tokenizer.texts_to_sequences(map(lambda s: ' '.join(s), val_cds))       # tokenising output validation data

train_tags_padded = pad_sequences(train_tags, maxlen=MAX_LEN)               # padding output training data to obatin sequences of length MAX_LEN = 2000
val_tags_padded = pad_sequences(val_tags, maxlen=MAX_LEN)                   # padding output validation data to obatin sequences of length MAX_LEN = 2000

# train_tags_padded = np.expand_dims(train_tags_padded, -1)
# val_tags_padded = np.expand_dims(val_tags_padded, -1)

print( train_tags_padded.shape, val_tags_padded.shape)

Unique tags: {'GAA': 1, 'AAA': 2, 'GAT': 3, 'AAT': 4, 'AAG': 5, 'ATT': 6, 'TTT': 7, 'CAA': 8, 'TTG': 9, 'TTA': 10, 'AAC': 11, 'TCT': 12, 'GGT': 13, 'GTT': 14, 'AGA': 15, 'ATG': 16, 'GAC': 17, 'GCT': 18, 'ACT': 19, 'GAG': 20, 'TCA': 21, 'TAT': 22, 'ATA': 23, 'TTC': 24, 'ACA': 25, 'CCA': 26, 'ATC': 27, 'GCA': 28, 'AGT': 29, 'TAC': 30, 'TCC': 31, 'CAT': 32, 'CCT': 33, 'CTA': 34, 'CTT': 35, 'ACC': 36, 'CAG': 37, 'GTA': 38, 'GCC': 39, 'GTC': 40, 'GGA': 41, 'GTG': 42, 'CTG': 43, 'TGG': 44, 'AGC': 45, 'GGC': 46, 'AGG': 47, 'TCG': 48, 'ACG': 49, 'TGT': 50, 'CAC': 51, 'CCC': 52, 'CGT': 53, 'GCG': 54, 'GGG': 55, 'CTC': 56, 'CCG': 57, 'TGC': 58, 'CGA': 59, 'CGC': 60, 'CGG': 61, 'TAA': 62, 'TGA': 63, 'TAG': 64, '__PADDING__': 0}
(5370, 2000) (1343, 2000)


In [12]:
from keras.layers import Dense, Input, LSTM
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import EarlyStopping
from keras.layers import Embedding, Bidirectional, Dropout

# early stopping is used to stop training the model when there is no improvement in validation loss for 10 epochs (defined by patience)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

# embedding layer is used to generate embedding vectors of length 100 for each unique amino acid token
embedding_layer = Embedding(VOCAB_SIZE, 100, input_length=MAX_LEN)

# input layer that defines the shape of input data 
sequence_input = Input(shape=(MAX_LEN,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

# bi-directional LSTM layer, where each output value is a probability distribution over 64 possible codons
x = Bidirectional(LSTM(64, return_sequences=True))(embedded_sequences)
# dropout layer randomly drops out 30% of the neurons to speedup training
x = Dropout(0.3)(x)
# dense layer, a hidden layer with 32 neuron units with relu activation function
x = Dense(32, activation='relu')(x)
# output layer with a dimensionality of 65, where each dimension gives a probability of occurrence of a unique codon
# softmax activation is used since the output is a multiclass classification with 65 possible categories
preds = Dense(len(tag_index), activation='softmax')(x)

blstm_model = Model(sequence_input, preds)
blstm_model.compile(loss='sparse_categorical_crossentropy',            
              optimizer='adam',
              metrics=['sparse_categorical_accuracy'])
blstm_model.summary()

# fitting the bi-directional model on Yeast training data and fine tuning on validation data, with a batch size of 32 and maximum number of epochs = 100
# earlystopping is used to avoid overfitting on training data
blstm_model.fit(train_sequences_padded, train_tags_padded,
          batch_size=32,
          epochs=100,
          validation_data=(val_sequences_padded, val_tags_padded),
          callbacks=[earlystopping])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2000)]            0         
_________________________________________________________________
embedding (Embedding)        (None, 2000, 100)         2100      
_________________________________________________________________
bidirectional (Bidirectional (None, 2000, 128)         84480     
_________________________________________________________________
dropout (Dropout)            (None, 2000, 128)         0         
_________________________________________________________________
dense (Dense)                (None, 2000, 32)          4128      
_________________________________________________________________
dense_1 (Dense)              (None, 2000, 65)          2145      
Total params: 92,853
Trainable params: 92,853
Non-trainable params: 0
__________________________________________________

In [13]:
# uploading test data obtained from Parts iGem library
from google.colab import files
from keras.preprocessing.sequence import pad_sequences

file = files.upload()
print(file)

# load Yeast data and filter to get only coding sequences
test_data = pd.read_csv('Mixed_CDS.csv')
# filter data to extract only sequences
test_df = test_data['Part Sequence']

test_aas = []
test_cds = []

for seq in test_df:
    cds = codon(seq.upper())                # convert nucleotide sequence to sequence of codons
    test_cds.append(cds)
    test_aas.append(translate(cds))         # translate codon sequence to a sequence of amino acids

print(len(test_aas))
print(len(test_cds))

Saving Mixed_CDS.csv to Mixed_CDS.csv
{'Mixed_CDS.csv': b'\xef\xbb\xbfID,Description,Part Type,Part Sequence\r\nBBa_C0011,"BBa_C0011 P 6291 Coding ""ccdB gene from %3CI%3E E. coli %3C/I%3E""",Coding,atgcagtttaaggtttacacctataaaagagagagccgttatcgtctgtttgtggatgtacagagtgatattattgacacgcccgggcgacggatggtgatccccctggccagtgcacgtctgctgtcagataaagtctcccgtgaactttacccggtggtgcatatcggggatgaaagctggcgcatgatgaccaccgatatggccagtgtgccggtctccgttatcggggaagaagtggctgatctcagccaccgcgaaaatgacatcaaaaacgccattaacctgatgttctggggaatataa\r\nBBa_C0012,"BBa_C0012 A 153 Coding ""lacI repressor  from E. coli (+LVA)""",Coding,atggtgaatgtgaaaccagtaacgttatacgatgtcgcagagtatgccggtgtctcttatcagaccgtttcccgcgtggtgaaccaggccagccacgtttctgcgaaaacgcgggaaaaagtggaagcggcgatggcggagctgaattacattcccaaccgcgtggcacaacaactggcgggcaaacagtcgttgctgattggcgttgccacctccagtctggccctgcacgcgccgtcgcaaattgtcgcggcgattaaatctcgcgccgatcaactgggtgccagcgtggtggtgtcgatggtagaacgaagcggcgtcgaagcctgtaaagcggcggtgcacaatcttctcgcgcaacgcgtcagtgggctgatcattaactatccgctggatgaccaggatgcca

In [14]:
# model testing

# tokenising amino acid test sequence using he same tokeniser as that used for training data
test_seq = words_tokenizer.texts_to_sequences(map(lambda s: ' '.join(s), test_aas)) 
# padding tokenised test sequence to convert it to a length of MAX_LEN = 2000
padded_test_sequence = pad_sequences(test_seq, maxlen=MAX_LEN)

# use the trained bi-directional LSTM model for making predictions on the test data
y_hat = blstm_model.predict(padded_test_sequence)

# prints the shape of output - 600 test sequences, of length 2000 each, 
# with an output of probability distribution over 65 values for each input (amino acid) token
print(np.asarray(y_hat).shape)

tot_prob = []
# greedy approach used to find the optimal codon, the one with highest probability vale for each input (amino acid) token
for ypred in y_hat:
    prob = []
    for i in ypred:
        prob.append(np.argmax(i))   # retrieves the corresponding codon token with highest probability value for each input (amino acid) token
    tot_prob.append(prob)

(600, 2000, 65)


In [15]:
# function to map each output token back to codon
yhat = []
def token_to_word(yhat):
    for word, index in tags_tokenizer.word_index.items():
        if index == yhat:
            out_word = word
            
            return out_word

# loop that iterates over each of the predicted test sequences to reverse map output tokens to codons          
for prob in tot_prob:
    y_hat_cds = []
    for i in prob:
        if i==0:
            continue
        else:
            y = token_to_word(i)
            y_hat_cds.append(y)
    yhat.append(y_hat_cds)

import statistics
cds_acc = []
aas_acc = []

# function to calculate accuracy of predicting correct synonymous codons for a given sequence of amino acids
def cal_acc(ytrue, ypred):
    count = 0
    min_len = min(len(ytrue), len(ypred))
    for i in range(min_len):
        if ytrue[i]==ypred[i]:
            count+=1

    return (count/len(ytrue))

# loop that iterates over each predicted test sequence to compute its accuracy of prediction
for i in range(len(test_cds)):
    yhat_aa = translate(yhat[i])
    ytrue_aa = translate(test_cds[i])
    cds_acc.append(cal_acc(test_cds[i], yhat[i]))
    aas_acc.append(cal_acc(ytrue_aa, yhat_aa))

print('Percentage of codons unaltered: ', statistics.mean(cds_acc))
print('Accuracy of predicting synonymous codons (same amino acids): ', statistics.mean(aas_acc))

Percentage of codons unaltered:  0.35934746605334955
Accuracy of predicting synonymous codons (same amino acids):  0.9347811166168231


In [16]:
from pickle import dump
# save the model to file
blstm_model.save('model.h5')

# save the tokenizers
dump(words_tokenizer, open('words_tokenizer.pkl', 'wb'))
dump(tags_tokenizer, open('tags_tokenizer.pkl', 'wb'))